<a href="https://colab.research.google.com/github/KamranAmjadAli/learning/blob/main/fine_tune_on_alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = 'drive/MyDrive/Datasets/'

In [ ]:
import pandas as pd
p_df = pd.read_parquet(data_path+'alpaca_data.parquet', engine='pyarrow')

In [ ]:
p_df.head()

,instruction,input,output,text
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...,Below is an instruction that describes a task....
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye...",Below is an instruction that describes a task....
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain...",Below is an instruction that describes a task....
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...,Below is an instruction that describes a task....
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...,Below is an instruction that describes a task....


In [ ]:
# !pip install tiktoken transformers peft datasets trl
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

In [ ]:
def train():
  train_dataset = Dataset.from_pandas(p_df, split="train")
  tokenizer = AutoTokenizer.from_pretrained("abhishek/llama-2-7b-hf-small-shards", trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  model = AutoModelForCausalLM.from_pretrained("abhishek/llama-2-7b-hf-small-shards", load_in_4bit=True, torch_dtype=torch.float16, device_map="auto")
  model.resize_token_embeddings(len(tokenizer))
  model = prepare_model_for_int8_training(model)
  peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
  model = get_peft_model(model, peft_config)
  training_args = TrainingArguments(output_dir = data_path+"7b-tuned-alpaca", per_device_train_batch_size=4, optim="adamw_torch",
                                    logging_steps=100, fp16=True, warmup_ratio=0.1, lr_scheduler_type="linear", num_train_epochs=1,
                                    save_strategy="epoch", push_to_hub=False)
  trainer = SFTTrainer(model=model, train_dataset=train_dataset, dataset_text_field="text", max_seq_length=1024, tokenizer=tokenizer,
                       args=training_args, packing=True, peft_config=peft_config)
  trainer.train()


In [ ]:
train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarni

Step,Training Loss


Step,Training Loss
100,1.450800
200,1.247700
300,1.066900
400,0.952500
500,0.925900
600,0.913500
